In [1]:
from util.settings import ReID
from util.reid_onnx_helper import ReidHelper
vehicleID = ReID('veriwild_dynamic.onnx')


In [2]:
import pandas as pd

dataset_path = 'dataset/car_appearance/'

df = pd.read_csv(dataset_path+"/annotation.csv",)


In [3]:
import cv2

In [4]:
# the group that more than one element
df_ = df.groupby('classId').apply(lambda group:group if len(group) > 1 else None).reset_index(drop=True)

In [5]:
anchors = df_.groupby("classId").first()

In [6]:
gallaries = df_.groupby('classId').apply(lambda group:group.iloc[1:]).reset_index(drop=True)

In [7]:
import onnxruntime as onnx
from util.util import calc_cosign_similarity, get_images
from tqdm import tqdm


In [8]:
reid = ReidHelper(vehicleID)
rank1 = 0
rank5 = 0
rank10 = 0
i = 0      # 실제 클래스의 수
for cid, anchor in tqdm(anchors.iterrows(), total=anchors.shape[0]):

    #query image
    anchor_img = get_images(dataset_path+anchor['file_name'])
    anchor_img_feat = reid.infer(anchor_img)

    #gallary image
    gallary_similarity = []
    for _cid, gallary in gallaries.groupby("classId").agg(pd.DataFrame.sample).iterrows():

        gallary_img = get_images(dataset_path+gallary['file_name'])
        gallary_img_feat = reid.infer(gallary_img)

        similarity = calc_cosign_similarity(anchor_img_feat, gallary_img_feat)

        gallary_similarity.append(
            tuple([similarity, _cid, gallary['file_name']]))

    gallary_similarity.sort(reverse=True)

    rank1 += 1 if list(filter(lambda x: x[1]
                       == cid, gallary_similarity[:1])) else 0
    rank5 += 1 if list(filter(lambda x: x[1]
                       == cid, gallary_similarity[:5])) else 0
    rank10 += 1 if list(filter(lambda x: x[1]
                        == cid, gallary_similarity[:10])) else 0
    i += 1
rank1 /= i
rank5 /= i
rank10 /= i

print(rank1, rank5, rank10)


100%|██████████| 1003/1003 [3:15:18<00:00, 11.68s/it] 

0.17746759720837488 0.3160518444666002 0.3898305084745763


In [9]:
rank1*i, rank5*i, rank10*i, i


(178.0, 317.0, 391.0, 1003)